In [2]:
import tensorflow as tf
keras=tf.keras
layers=keras.layers

In [3]:
keras.optimi

AttributeError: module 'tensorflow.keras' has no attribute 'optimi'

In [2]:
class ConvBlock(layers.Layer):
    def __init__(self,units,kenral_size,strides,use_activation=True):
        super(ConvBlock,self).__init__()
        self.conv2d=layers.Conv2D(units,kernel_size=kenral_size,strides=strides,padding='same')
        self.bn=layers.BatchNormalization()
        self.ua=use_activation
    def call(self,input):
        x=self.conv2d(input)
        x=self.bn(x)
        if self.ua==True:
            x=tf.nn.relu(x)
        return x


In [3]:
c1=ConvBlock(64,3,2)


In [4]:
input=keras.Input(shape=(224,224,3))
c1(input)

<KerasTensor: shape=(None, 112, 112, 64) dtype=float32 (created by layer 'conv_block')>

In [5]:
class DetailBranch(layers.Layer):
    def __init__(self):
        super(DetailBranch,self).__init__()
        self.s1Conv1=ConvBlock(units=64,kenral_size=3,strides=2)
        self.s1Conv2=ConvBlock(units=64,kenral_size=3,strides=1)
        self.s2Conv1=ConvBlock(units=64,kenral_size=3,strides=2)
        self.s2Conv2=ConvBlock(units=64,kenral_size=3,strides=1)
        self.s2Conv3=ConvBlock(units=64,kenral_size=3,strides=1)
        self.s3Conv1=ConvBlock(units=128,kenral_size=3,strides=2)
        self.s3Conv2=ConvBlock(units=128,kenral_size=3,strides=1)
        self.s3Conv3=ConvBlock(units=128,kenral_size=3,strides=1)
    def call(self,input):
        x=self.s1Conv1(input)
        x=self.s1Conv2(x)
        x=self.s2Conv1(x)
        x=self.s2Conv2(x)
        x=self.s2Conv3(x)
        x=self.s3Conv1(x)
        x=self.s3Conv2(x)
        x=self.s3Conv3(x)
        return x

In [6]:

db=DetailBranch()


In [7]:
in1=keras.Input(shape=(256,256,3))
db(in1)

<KerasTensor: shape=(None, 32, 32, 128) dtype=float32 (created by layer 'detail_branch')>

In [8]:
class StemBlock(layers.Layer):
    def __init__(self,channels=16):
        super(StemBlock,self).__init__()
        self.conv1=ConvBlock(units=channels,kenral_size=3,strides=2)
        self.conv2=ConvBlock(units=channels//2,kenral_size=1,strides=1)
        self.conv3=ConvBlock(units=channels,kenral_size=3,strides=2)
        self.conv4=ConvBlock(units=channels,kenral_size=3,strides=1)
        self.maxpool=layers.MaxPool2D(pool_size=3,strides=2,padding='same')
    def call(self,input):
        x=self.conv1(input)
        x1=self.maxpool(x)
        x2=self.conv2(x)
        x2=self.conv3(x2)
        x3=tf.concat([x2,x1],axis=-1)
        x3=self.conv4(x3)
        return x3


In [9]:
s1=StemBlock()
s1(in1)

<KerasTensor: shape=(None, 64, 64, 16) dtype=float32 (created by layer 'stem_block')>

In [56]:
class DWConv(layers.Layer):
    def __init__(self,kernel_size,strides,e=1):
        super(DWConv,self).__init__()
        self.dwconv=layers.DepthwiseConv2D(kernel_size=kernel_size,strides=strides,depth_multiplier=e,padding='same')
        self.bn=layers.BatchNormalization()
    def call(self,input):
        x=self.dwconv(input)
        return self.bn(x)



<KerasTensor: shape=(None, 112, 112, 6) dtype=float32 (created by layer 'dw_conv_1')>

In [35]:
class GatherExpansion(layers.Layer):
    def __init__(self,units,expansion_ration,layers_name='',strides=2):
        super(GatherExpansion,self).__init__()
        self.conv1=ConvBlock(units=units,kenral_size=3,strides=1)
        self.conv2=ConvBlock(units=units,kenral_size=1,strides=1,use_activation=False)
        self.conv3=ConvBlock(units=units,kenral_size=1,strides=1,use_activation=False)
        self.dwconv1=DWConv(kernel_size=3,
                            strides=strides,
                            e=expansion_ration)
        self.dwconv2=DWConv(kernel_size=3,
                            strides=1,
                            e=1)
        self.dwconv3=DWConv(kernel_size=3,
                            strides=strides,
                            e=1)
        self.relu=layers.ReLU(name=layers_name)                    
    def call(self,input):
        x=self.conv1(input)
        x1=self.dwconv1(x)
        x1=self.dwconv2(x1)
        x1=self.conv2(x1)
        x2=self.dwconv3(input)
        x2=self.conv3(x2)
        x3=tf.add(x1,x2)      
        return self.relu(x3)


        

In [12]:
class SegHead(layers.Layer):
    def __init__(self,units,numclasses,size):
        super(SegHead,self).__init__()
        self.conv1=ConvBlock(units=units,kenral_size=3,strides=1)
        self.conv2=ConvBlock(units=numclasses,kenral_size=1,strides=1,use_activation=False)
        self.up=layers.UpSampling2D(size,interpolation='bilinear')
    def call(self,input):
        x1=self.conv1(input)
        x1=self.conv2(x1)
        x1=self.up(x1)
        return x1

In [4]:
x1=SegHead(16,34,4)(x1)
x2=SegHead(32,34,8)(x2)
x3=SegHead(64,34,16)(x3)
x4=SegHead(128,34,32)(x4)


NameError: name 'SegHead' is not defined

In [13]:
class ContextEmbelding(layers.Layer):
    def __init__(self,units):
        super(ContextEmbelding,self).__init__()
        self.conv1=ConvBlock(units,kenral_size=1,strides=1)
        self.conv2=ConvBlock(units,kenral_size=3,strides=1)
    def call(self,input):
        x=tf.reduce_mean(input,axis=[1,2],keepdims=True)
        x=layers.BatchNormalization()(x)
        x=self.conv1(x)
        x1=tf.add(input,x)
        x1=self.conv2(x1)
        return x1


In [14]:
c1=ContextEmbelding(3)
c1(input)

<KerasTensor: shape=(None, 224, 224, 3) dtype=float32 (created by layer 'context_embelding')>

In [43]:
input1=layers.Input(shape=(1024,2048,3))
s1=StemBlock()
x1=s1(input1)
x1

<KerasTensor: shape=(None, 256, 512, 16) dtype=float32 (created by layer 'stem_block_2')>

In [44]:
ge1=GatherExpansion(units=32,expansion_ration=6,layers_name='ge1',strides=2)
x2=ge1(x1)
x2

<KerasTensor: shape=(None, 128, 256, 32) dtype=float32 (created by layer 'gather_expansion_24')>

In [45]:
ge2=GatherExpansion(units=32,expansion_ration=6,layers_name='ge2',strides=1)
x2=ge2(x2)
x2

<KerasTensor: shape=(None, 128, 256, 32) dtype=float32 (created by layer 'gather_expansion_25')>

In [46]:
ge3=GatherExpansion(units=64,expansion_ration=6,layers_name='ge3',strides=2)
ge4=GatherExpansion(units=64,expansion_ration=6,layers_name='ge4',strides=1)
x3=ge3(x2)
x3=ge4(x3)
x3

<KerasTensor: shape=(None, 64, 128, 64) dtype=float32 (created by layer 'gather_expansion_27')>

In [47]:
ge5=GatherExpansion(units=128,expansion_ration=6,layers_name='ge5',strides=2)
ge6=GatherExpansion(units=128,expansion_ration=6,layers_name='ge6',strides=1)
ge7=GatherExpansion(units=128,expansion_ration=6,layers_name='ge7',strides=1)
ge8=GatherExpansion(units=128,expansion_ration=6,layers_name='ge8',strides=1)

x4=ge5(x3)
x4=ge6(x4)
x4=ge7(x4)
x4=ge8(x4)
x4

<KerasTensor: shape=(None, 32, 64, 128) dtype=float32 (created by layer 'gather_expansion_31')>

In [49]:
model1=keras.models.Model(inputs=input1,outputs=[x1,x2,x3,x4])

In [70]:
class BiSeNetV2(keras.Model):
    def __init__(self,numclasses=34):
        super(BiSeNetV2,self).__init__()
        self._DetailBranch=DetailBranch()
        self.model1=model1
        self.FeatureFusion=FeatureFusion()
        self.contex=ContextEmbelding(128)
        self.up1=layers.UpSampling2D(size=8,interpolation='bilinear')
    def call(self,input):
        DB_input=self._DetailBranch(input)
        x1,x2,x3,x4=self.model1(input)
        SB_input=self.contex(x4)
        result=self.FeatureFusion(DB_input,SB_input)
        result=self.up1(result)
        return result



In [69]:
class FeatureFusion(layers.Layer):
    def __init__(self,units=128,numclasses=34):
        super(FeatureFusion,self).__init__()
        self.dwconv1=DWConv(kernel_size=3,strides=1)
        self.conv1=ConvBlock(units=units,kenral_size=3,strides=2,use_activation=False)
        self.conv2=layers.Conv2D(units,kernel_size=1,strides=1,padding='same')
        self.avgpool=layers.AveragePooling2D(pool_size=3,strides=2,padding='same')
        self.conv3=ConvBlock(units=units,kenral_size=3,strides=1,use_activation=False)
        self.dwconv2=DWConv(kernel_size=3,strides=1)
        self.up1=layers.UpSampling2D(size=4,interpolation='bilinear')
        self.up2=layers.UpSampling2D(size=4,interpolation='bilinear')
        self.conv4=ConvBlock(units=units,kenral_size=1,strides=1,use_activation=False)
        self.conv5=ConvBlock(units=numclasses,kenral_size=3,strides=1,use_activation=False)
    def call(self,DB_input,SB_input):
        x1=self.dwconv1(DB_input)
        x1=self.conv2(x1)
        x2=self.conv1(DB_input)
        x2=self.avgpool(x2)
        x3=self.conv3(SB_input)
        x3=self.up1(x3)
        x3=tf.nn.sigmoid(x3)
        x4=self.dwconv2(SB_input)
        x4=self.conv4(x4)
        x4=tf.nn.sigmoid(x4)
        x=tf.multiply(x1,x3)
        y=tf.multiply(x2,x4)
        y=self.up2(y)
        out=tf.add(x,y)
        out=self.conv5(out)
        return out


In [65]:
in11=keras.Input(shape=(256,256,3))
in22=keras.Input(shape=(64,64,3))
f1=FeatureFusion()
f1(in11,in22)

<KerasTensor: shape=(None, 256, 256, 34) dtype=float32 (created by layer 'feature_fusion_3')>

In [54]:
256/4

64.0

In [71]:
in11=keras.Input(shape=(1024,2048,3))
bis1=BiSeNetV2()
bis1(in11)

<KerasTensor: shape=(None, 1024, 2048, 34) dtype=float32 (created by layer 'bi_se_net_v2')>

In [ ]:
def rocket(input,units,size,numclasses=34,x):
    model=keras.models.Model(inputs=input1,outputs=x)
    with tf.GradientTape() as t:
        pred=model(input)
        pred=SegHead(units,numclasses,size)(pred)
        loss1=loss(labels,x1)
    gradies=t.gradient(loss1,net.trainable_variables)#求解梯度
    optimizer.apply_gradients(zip(gradies,net.trainable_variables))#


In [ ]:
keras.models.Model(inputs=input1,outputs=x1)